In [152]:
import pandas as pd


In [153]:
nodes=pd.read_csv('nodes_df.csv')

In [154]:
nodes=nodes.set_index('id')

In [155]:
edges=pd.read_csv('edges_df.csv')

In [159]:
import networkx as nx
G=nx.Graph()

In [160]:
for ind, row in edges.iterrows():
    G.add_edge(row['from'], row['to'])

In [161]:
components=[c for c in sorted(nx.connected_components(G), key=len, reverse=True)]

In [162]:
# 
for c in components:
     print(len(c))

66513


In [163]:
# all_disconnected_nodes=[]
# for c in components[1:]:
#     all_disconnected_nodes.extend(list(c))

In [164]:
nodes['component']=0
for i, c in enumerate(components):
    nodes.loc[list(c), 'component']='c'+str(i)

# Create Fake WGS model

In [165]:
import numpy as np
utm_zone = int(np.floor((-71 + 180) / 6) + 1)
crs = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"

In [166]:
import geopandas as gpd
nodes_gdf=gpd.GeoDataFrame(data=nodes, geometry=gpd.points_from_xy(
    x=nodes['x'], y=nodes['y'], z=nodes['z'], crs=crs))

In [167]:
nodes_gdf_wgs_fake=nodes_gdf.to_crs('EPSG:4326')

In [168]:
nodes_gdf_wgs_fake.to_file('nodes_gdf_wgs_fake.geojson',driver='GeoJSON')

In [169]:
centroids=nodes_gdf_wgs_fake.geometry.centroid
nodes_gdf_wgs_fake['lon']=[c.x for c in centroids]
nodes_gdf_wgs_fake['lat']=[c.y for c in centroids]

C:\Users\adminlocal\AppData\Local\Temp\ipykernel_22144\1538748961.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids=nodes_gdf_wgs_fake.geometry.centroid


In [ ]:
edges=edges.merge(nodes_gdf_wgs_fake[['lon', 'lat', 'z']], 
                  left_on='from', right_index=True,
                 how='left').rename(columns={ 'lon': 'from_lon', 'lat': 'from_lat', 'z': 'from_z'})

edges=edges.merge(nodes_gdf_wgs_fake[['lon', 'lat', 'z']], 
                  left_on='to', right_index=True,
                 how='left').rename(columns={ 'lon': 'to_lon', 'lat': 'to_lat', 'z': 'to_z'})

In [ ]:
edges[['from_lon','from_lat','from_z','to_lon','to_lat','to_z','id']
     ].to_csv('edges_fake_wgs.csv')